# Paper: Ocean mixing in a shelf seas driven by energetic nonlinear internal waves

Prepared for Journal of Geophysical Research: Oceans \
Notebook author: Chris A Whiwtwell \
Contact: chris.a.whitwell@outlook.com \ 

This notebook contains the analysis to perform site characterisation and internal wave identification for one site (Example here is for 200m deep mooring). All imported modules are pip installable.

In [ ]:
import proplot as pplt

from scipy import signal,interpolate
import xarray as xr
import numpy as np
from copylot.signals import isoslice
import wmtsa.modwt

import iwaves
from iwaves.kdv.vkdv import vKdV as KdV
from iwaves import IWaveModes
from iwaves.utils.density import FitDensity
from scipy.interpolate import PchipInterpolator
import scipy.linalg as la

GRAV = 9.81
RHO0 = 1024.

def fit_bmodes_linear(rho, rhoz, z,  zmin, modes,\
        Nz=100, density_func='single_tanh', full_output=False):
    """
    Compute the linear modal amplitude to the mode numbers in the list

    Inputs:
    ---
        rho - matrix[nz, nt], density data
        rhoz - vector[nz], background density profile from bottom to top (desceding)
        z - vector[nz], depth from bottom to top, negative values (ascending)
        modes - list[nmodes], mode numbers in python index i.e. 0=1

    Returns
    ---
        A_t - matrix[nmodes, nt], modal amplitude
        phi - matrix[nmodes, nz], modal structure functions
        rhofit - matrix[nz, nt], best fit density profile

    """

    nz, nt = rho.shape
    nmodes = len(modes)

    # Compute buoyancy from density and backgroud density
    rhopr = rho.T - rhoz[np.newaxis,...]
    b = GRAV*rhopr/RHO0

    # Compute the modal structures
    L = np.zeros((nz,nmodes))
    phi_n = []
    c = []
    alpha = []
    beta = []
    # Calculate dz
    Z = np.linspace(zmin, 0, Nz)
    dz = np.mean(np.diff(Z))

    for ii, mode in enumerate(modes):
        # Use the mode class to create the profile
        iw = IWaveModes(rhoz, z,\
                density_class=FitDensity, density_func=density_func)
        phi, c1, he, znew = iw(zmin, dz, mode)
        c.append(c1)
        alpha.append(calc_alpha(phi,c1,-np.mean(np.diff(znew))))
        beta.append(calc_beta(phi,c1,-np.mean(np.diff(znew))))
        if full_output:
            if ii==0:
                Nz = iw.Z.size
                Lout = np.zeros((Nz, nmodes))
            Lout[:,ii] = phi * iw.N2
            phi_n.append(phi)

        ## Interpolate the modal shape and N2 onto the measured depth locations
        F = PchipInterpolator(iw.Z[::-1], iw.phi[::-1])
        my_phi = F(z)

        F = PchipInterpolator(iw.Z[::-1], iw.N2[::-1])
        my_N2 = F(z)

        L[:,ii] = my_phi*my_N2
        phi_n.append(my_phi)


    ## Fit Ax=b
    A_t,_,_,_ = la.lstsq(L , b.T)

    # Reconstruct the density field
    bfit_n = L[:,np.newaxis,:]*A_t.T[np.newaxis,...]
    bfit = bfit_n.sum(axis=-1) # sum the modes

    rhoprfit = bfit.T*RHO0/GRAV
    rhofit = rhoprfit + rhoz[np.newaxis,:]

    if full_output:
        bfit_n = Lout[:,np.newaxis,:]*A_t.T[np.newaxis,...]
        bfit = bfit_n.sum(axis=-1) # sum the modes
        rhoprfit = bfit.T*RHO0/GRAV
        rhofit_full = rhoprfit + iw.rhoZ[np.newaxis,:]
        return A_t, np.array(phi_n).T, rhofit, rhofit_full, iw,np.array(c)
    else:
        return A_t, np.array(phi_n).T, rhofit,np.array(c),np.array(alpha),np.array(beta)
    
def calc_alpha(phi, c, dz):
    phi_z = np.gradient(phi,-dz)
    num = 3*c*np.trapz( phi_z**3., dx=dz)
    den = 2*np.trapz( phi_z**2., dx=dz)

    return num/den

def calc_beta(phi, c, dz):
    phi_z = np.gradient(phi, dz)
    num = c*np.trapz( phi**2., dx=dz)
    den = 2*np.trapz( phi_z**2., dx=dz)

    return num/den

%matplotlib inline

# Load in all the netcdfs

In [ ]:
### ADCP 
path_ADCP= r"C:\Users\21310917\OneDrive - The University of Western Australia\Data\RS2019\RowleyShoals2019_RDIADCPData_float64.nc"
ds_adcp =  xr.open_dataset(path_ADCP,group='T200_RDI_75kHz_LR_24613').sel(time=slice('2019-03-07','2019-04-05 00:00')).sel(distance=slice(0,170))

### Temperature
path = r"C:\Users\21310917\OneDrive - The University of Western Australia\Data\RS2019\RowleyShoals_Gridded_Mooring_T_20sec.nc"
ds_T = xr.open_dataset(path,group='T200').sel(time=slice('2019-03-08 00','2019-04-05 00:00')).sel(depth=slice(10,200))


### Heat Flux
ds_fs = xr.open_dataset(r"..\Data\New\RS2019_T200_FS_20sec Shear20 Temp10.nc")
ds_fs = ds_fs.where(ds_fs['JQ'] > 10**-3).where(ds_fs['JQ'] < 10**5)


### Convert datum to surface
ds_fs['depth'] = ds_fs['depth'] - 200
ds_T['depth'] = ds_T['depth'] - 200

# Process Temperature Data

In [ ]:
##### Define filter + density conversion
filt_3d_lp =  signal.butter(4,1/(72*3600),output='sos',fs=1/20)
filt_2h_lp =  signal.butter(4,1/(2*3600),output='sos',fs=1/20)

slope = -0.24907119420412366
intercept = 1030.3291931675537

##### T200 
ds_T['rho'] = slope*ds_T['Temperature'] + intercept # convert temp to density with linear EoS
ds_T['rho_3d_lp'] = xr.DataArray(signal.sosfiltfilt(filt_3d_lp,ds_T['rho'].values,axis=-1),dims=['depth','time'],coords={'depth':ds_T.depth.values,'time':ds_T.time.values}) # 3 day lowpass
ds_T['rho_2h_lp'] = xr.DataArray(signal.sosfiltfilt(filt_2h_lp,ds_T['rho'].values,axis=-1),dims=['depth','time'],coords={'depth':ds_T.depth.values,'time':ds_T.time.values}) # 2 hour lowpass

ds_T['N_3d_lp'] =  np.sqrt(-9.81*ds_T['rho_3d_lp'].differentiate('depth')/ds_T['rho_3d_lp'])
ds_T['N_2h_lp'] =  np.sqrt(-9.81*ds_T['rho_2h_lp'].differentiate('depth')/ds_T['rho_2h_lp'])

# Process ADCP Data

In [ ]:
filt = signal.butter(2,1/(6*3600),btype='lowpass',output='sos',fs=1/60)

u = xr.DataArray(signal.sosfiltfilt(filt,ds_adcp['u'].mean('distance').values),dims=['time'],coords={'time':ds_adcp.time.values})
v = xr.DataArray(signal.sosfiltfilt(filt,ds_adcp['v'].mean('distance').values),dims=['time'],coords={'time':ds_adcp.time.values})

theta = -55 # this is cross shelf at RS2019
cross_bt = u*np.cos(np.radians(theta)) + v*np.sin(np.radians(theta))

# Calculate Internal Wave Characteristics

In [ ]:
#### Internal wave characterisation
timedelta = np.timedelta64(24,'h')
time = np.datetime64('2019-03-08 00:00')

m1_alpha,m1_beta = [],[]
m2_alpha,m2_beta = [],[]
m1_cs,m2_cs = [],[]
A1,A2 = [],[]
u,l,m,t = [],[],[],[]
Nn = []

times = []

for day in range(28):
    tslice = slice(time,time+timedelta)
    
    #########
    ### T200
    sub = ds_T.sel(time=tslice).sel(depth=slice(-190,0))    
    depth = -200
    
    rho = sub['rho'].values
    rhoz = sub['rho_3d_lp'].mean('time').values
    z = sub['depth'].values
    z_2 = np.linspace(min(z),max(z),100)
    
    A,phi,rho_out,c,alphas,betas = fit_bmodes_linear(rho,rhoz,z,zmin=depth,modes=[0,1],density_func='double_tanh_new')
    A1.append(A[0,:])
    A2.append(A[1,:])
    m1_cs.append(c[0])
    m2_cs.append(c[1])

    m1_alpha.append(alphas[0])
    m1_beta.append(betas[0])
    m2_alpha.append(alphas[1])
    m2_beta.append(betas[1])
    
        
    fit_1 = np.polynomial.chebyshev.Chebyshev.fit(z,phi[:,0],8)    
    high_res_1 = fit_1(z_2)
    max1 = np.interp(z_2[np.argmax(high_res_1)],z,rhoz)
    m1_iso = (max1-intercept)/slope

    fit = np.polynomial.chebyshev.Chebyshev.fit(z,phi[:,1],8)    
    high_res = fit(z_2)
    max_iso_2 = np.interp(z_2[np.argmax(high_res)],z,rhoz)
    min_iso_2 = np.interp(z_2[np.argmin(high_res)],z,rhoz)
    m2_iso_1=(min(max_iso_2,min_iso_2)-intercept)/slope
    m2_iso_2=(max(max_iso_2,min_iso_2)-intercept)/slope
        
    m,u,l,N = [],[],[],[]
    for t in range(len(sub.time.values)):
        u.append(isoslice(sub['depth'].values[:,None],sub['Temperature'].values[:,t][:,None],isoval=m2_iso_1,masking=False))
        l.append(isoslice(sub['depth'].values[:,None],sub['Temperature'].values[:,t][:,None],isoval=m2_iso_2,masking=False))
        N.append(isoslice(sub['N_2h_lp'].values[:,t][:,None],sub['Temperature'].values[:,t][:,None],isoval=m1_iso,masking=False))
    Nn.append(np.asarray(N).flatten())
    u.append(np.asarray(u).flatten())
    l.append(np.asarray(l).flatten())
    t.append(sub.time.values)
      
    times.append(time + timedelta/2)
    time += timedelta

times = np.asarray(times)
  
### 200
u = np.asarray(u).flatten()
l = np.asarray(l).flatten()
Nn = np.asarray(Nn).flatten()
t = np.asarray(t).flatten()
A1 = np.asarray(A1).flatten()
A2 = np.asarray(A2).flatten()

m1_cs = np.asarray(m1_cs)
m2_cs = np.asarray(m2_cs)
m1_alpha = np.asarray(m1_alpha)
m1_beta = np.asarray(m1_beta)
m2_alpha = np.asarray(m2_alpha)
m2_beta = np.asarray(m2_beta)

##################
### Add to Dataset
### 200
_,index = np.unique(t, return_index=True)

u = xr.DataArray(u,dims=['time'],coords={'time':t})
u = u.isel(time=index)
ds_T['m2_upper_iso'] = u.interp_like(ds_T['Temperature'].mean('depth'))
l = xr.DataArray(l,dims=['time'],coords={'time':t})
l = l.isel(time=index)
ds_T['m2_lower_iso'] = l.interp_like(ds_T['Temperature'].mean('depth'))

Nn = xr.DataArray(Nn,dims=['time'],coords={'time':t})
Nn = Nn.isel(time=index)
ds_T['m1_iso_N'] = Nn.interp_like(ds_T['Temperature'].mean('depth'))

A1 = xr.DataArray(A1,dims=['time'],coords={'time':t},attrs={'long_name':'Mode-1 Modal Amplitude Fitting'})
A1 = A1.isel(time=index)
ds_T['A1_MAF'] = A1.interp_like(ds_T['Temperature'].mean('depth'))

A2 = xr.DataArray(A2,dims=['time'],coords={'time':t},attrs={'long_name':'Mode-2 Modal Amplitude Fitting'})
A2 = A2.isel(time=index)
ds_T['A2_MAF'] = A2.interp_like(ds_T['Temperature'].mean('depth'))

## Daily 
ds_T['time2'] =  xr.DataArray(times,dims=['time2'])
ds_T['alpha1'] = xr.DataArray(m1_alpha,dims=['time2'],coords={'time2':times})
ds_T['beta1'] = xr.DataArray(m1_beta,dims=['time2'],coords={'time2':times})
ds_T['alpha2'] = xr.DataArray(m2_alpha,dims=['time2'],coords={'time2':times})
ds_T['beta2'] = xr.DataArray(m2_beta,dims=['time2'],coords={'time2':times})
ds_T['c1'] = xr.DataArray(m1_cs,dims=['time2'],coords={'time2':times})
ds_T['c2'] = xr.DataArray(m2_cs,dims=['time2'],coords={'time2':times})

# Time-frequency decomposition of isotherm excursions

In [ ]:
### Fill gaps
ds_T['A1_MAF'] = ds_T['A1_MAF'].interpolate_na(dim='time',method='nearest',fill_value='extrapolate')
ds_T['A2_MAF'] = ds_T['A2_MAF'].interpolate_na(dim='time',method='nearest',fill_value='extrapolate')
ds_T['m2_upper_iso'] = ds_T['m2_upper_iso'].where(ds_T['m2_upper_iso'] < -20).interpolate_na(dim='time',method='nearest',fill_value='extrapolate')
ds_T['m2_lower_iso'] = ds_T['m2_lower_iso'].where(ds_T['m2_lower_iso'] < -20).interpolate_na(dim='time',method='nearest',fill_value='extrapolate')

# Wavelet transform on the mode 1 isotherm data
### 200
w,v = wmtsa.modwt.modwt(ds_T['A1_MAF'].values,wtf='la16')
J0 = w.info['J0']
js = np.arange(1,J0+1)
j_low = 20*(2**js)
j_high = 20*(2**(js+1))
jl = xr.DataArray(np.asarray(j_low),dims=['js'],coords={'js':js})
jh = xr.DataArray(np.asarray(j_high),dims=['js'],coords={'js':js})          
wr,vr = wmtsa.modwt.cir_shift(w,v,subtract_mean_VJ0t=True)
A1_wj =  xr.DataArray(np.asarray(wr),dims=['js','time'],
                                coords={'js':js,'time':ds_T.time.values})
A1_smooth = np.asarray(wmtsa.modwt.imodwt_smooth(v))
A1_details = np.asarray(wmtsa.modwt.imodwt_details(w))
A1_smooth =  xr.DataArray(np.asarray(A1_smooth)[:len(ds_T.time.values)],dims=['time'],
                               coords={'time':ds_T.time.values})
A1_details =  xr.DataArray(np.asarray(A1_details)[:,:len(ds_T.time.values)],dims=['js','time'],
                                coords={'js':js,'time':ds_T.time.values})

# A2
w,v = wmtsa.modwt.modwt(ds_T['A2_MAF'].values,wtf='la16')
wr,vr = wmtsa.modwt.cir_shift(w,v,subtract_mean_VJ0t=True)
A2_wj =  xr.DataArray(np.asarray(wr),dims=['js','time'],
                                coords={'js':js,'time':ds_T.time.values})
A2_smooth = np.asarray(wmtsa.modwt.imodwt_smooth(v))
A2_details = np.asarray(wmtsa.modwt.imodwt_details(w))
A2_smooth =  xr.DataArray(np.asarray(A2_smooth)[:len(ds_T.time.values)],dims=['time'],
                               coords={'time':ds_T.time.values})
A2_details =  xr.DataArray(np.asarray(A2_details)[:,:len(ds_T.time.values)],dims=['js','time'],
                                coords={'js':js,'time':ds_T.time.values})


# Bore Calculation and Identification

In [ ]:
filt_hp = signal.butter(2,[1/(30*60)],btype='highpass',output='sos',fs=1/20)
bore_thresh = 0.5
average_window = 15

bottom = xr.open_dataset(path,group='T200').sel(time=slice('2019-03-08','2019-04-05 00:00')).sel(depth=slice(0,20)).mean('depth')

bottom_a = signal.sosfiltfilt(filt_hp,bottom['Temperature'].values)
bottom_hb = np.abs(signal.hilbert(signal.sosfiltfilt(filt_hp,bottom['Temperature'].values)))
bottom_a = xr.DataArray(bottom_a,dims=['time'],coords={'time':bottom.time.values})
bottom_hb = xr.DataArray(bottom_hb,dims=['time'],coords={'time':bottom.time.values})


## Bore Contrib
# 200 - I had to pick out some times where mode-2 waves were identified as bores
mode2_mask = (ds_fs.time.values > np.datetime64('2019-04-01 12:00'))&(ds_fs.time.values < np.datetime64('2019-04-01 14:00'))
mode2_mask += (ds_fs.time.values > np.datetime64('2019-03-31 02:00'))&(ds_fs.time.values < np.datetime64('2019-03-31 04:00'))
mode2_mask += (ds_fs.time.values > np.datetime64('2019-03-30 07:00'))&(ds_fs.time.values < np.datetime64('2019-03-30 09:00'))
mode2_mask += (ds_fs.time.values > np.datetime64('2019-03-16 17:00'))&(ds_fs.time.values < np.datetime64('2019-03-16 18:30'))

ds_fs['Temp_Dave'] = ds_fs['Temperature'].mean('depth')
ds_fs['bottom_hb'] = bottom_hb.interp_like(ds_fs['Temp_Dave']).rolling(time=average_window,center=True,min_periods=1).mean()
ds_fs['bore_mask'] = (ds_fs['bottom_hb'] > bore_thresh).rolling(time=30,center=True,min_periods=1).max().astype(bool)&~mode2_mask
ds_fs['bore_mask2'] = (ds_fs['bottom_hb'] > bore_thresh).rolling(time=40,center=True,min_periods=1).max().astype(bool)&~mode2_mask

# Mode-1 and Mode-2 Amplitude and Wave Identification

In [ ]:
mode_2_false_id = (ds_fs.time.values > np.datetime64('2019-03-13 00:00'))&(ds_fs.time.values < np.datetime64('2019-03-13 04:00'))
mode_2_false_id += (ds_fs.time.values > np.datetime64('2019-03-19 20:00'))&(ds_fs.time.values < np.datetime64('2019-03-19 23:00'))
mode_2_false_id += (ds_fs.time.values > np.datetime64('2019-03-25 12:00'))&(ds_fs.time.values < np.datetime64('2019-03-25 16:00'))
mode_2_false_id += (ds_fs.time.values > np.datetime64('2019-03-26 00:00'))&(ds_fs.time.values < np.datetime64('2019-03-26 04:00'))
mode_2_false_id += (ds_fs.time.values > np.datetime64('2019-03-27 00:00'))&(ds_fs.time.values < np.datetime64('2019-03-27 04:00'))

In [ ]:
mode_1_LF_thresh = 7.5
mode_1_HF_thresh = 1.5
mode_2_LF_thresh = 7.5
filt = signal.butter(2,[1/(2*3600),1/600],btype='bandpass',output='sos',fs=1/20)
phase_upper_thresh = 2
phase_lower_thresh = -2
max_window = 15

### 200
m1_LF_band = (jl.values[:,None] > (2*np.pi/ds_T['m1_iso_N'].values))&((jl.values[:,None] < ((2*3600))))
m2_band = (jl.values[:,None] > 600)&((jl.values[:,None] < ((2*3600))))
HF_band = (jl.values[:,None] < (2*np.pi/ds_T['m1_iso_N'].values))&((jh.values[:,None] > (2*np.pi/ds_T['m1_iso_N'].values)))

### Hilbert transforms
# Mode 1 Low Frequency Calc
ds_fs['A1_LF'] =  A1_details.where(m1_LF_band).sum('js').interp_like(ds_fs['Temp_Dave'])
nan_mask = np.isnan(ds_fs['A1_LF'].values)
ds_fs['A1_LF_hb'] = xr.DataArray(np.abs(signal.hilbert(ds_fs['A1_LF'].values[~nan_mask])),dims=['time'],coords={'time':ds_fs.time.values[~nan_mask]}).interp_like(ds_fs['Temp_Dave'])
ds_fs['A1_LF_hb'] = ds_fs['A1_LF_hb'].rolling(time=average_window,center=True,min_periods=1).mean()

# Mode 1 High Frequency Calc
ds_fs['A1_HF'] =  A1_details.where(HF_band).sum('js').interp_like(ds_fs['Temp_Dave'])
nan_mask = np.isnan(ds_fs['A1_HF'].values)
ds_fs['A1_HF_hb'] = xr.DataArray(np.abs(signal.hilbert(ds_fs['A1_HF'].values[~nan_mask])),dims=['time'],coords={'time':ds_fs.time.values[~nan_mask]}).interp_like(ds_fs['Temp_Dave'])
ds_fs['A1_HF_hb'] = ds_fs['A1_HF_hb'].rolling(time=average_window,center=True,min_periods=1).mean()

# Mode 2 Calc
ds_fs['A2_LF'] =  A2_details.where(m2_band).sum('js').interp_like(ds_fs['Temp_Dave'])
nan_mask = np.isnan(ds_fs['A2_LF'].values)
ds_fs['A2_LF_hb'] = xr.DataArray(np.abs(signal.hilbert(ds_fs['A2_LF'].values[~nan_mask])),dims=['time'],coords={'time':ds_fs.time.values[~nan_mask]}).interp_like(ds_fs['Temp_Dave'])
ds_fs['A2_LF_hb'] = ds_fs['A2_LF_hb'].rolling(time=average_window,center=True,min_periods=1).mean()

### Largest wave mask -  if multiple waves are present, take largest amplitude
mode_1_LF_mask = (ds_fs['A1_LF_hb'] > ds_fs['A2_LF_hb'])&(ds_fs['A1_LF_hb'] > ds_fs['A1_HF_hb'])
mode_2_LF_mask = (ds_fs['A2_LF_hb'] > ds_fs['A1_LF_hb'])&(ds_fs['A2_LF_hb'] > ds_fs['A1_HF_hb'])
mode_1_HF_mask = ((ds_fs['A1_HF_hb'] > ds_fs['A1_LF_hb'])|(ds_fs['A1_LF_hb'] < mode_1_LF_thresh))&((ds_fs['A1_HF_hb'] > ds_fs['A2_LF_hb'])|(ds_fs['A2_LF_hb'] < mode_2_LF_thresh))

### Mode 1 ID
mode_1_mask = ((ds_fs['A1_LF_hb'] > mode_1_LF_thresh)).rolling(time=max_window,min_periods=5,center=True).max().astype(bool)&(~ds_fs['bore_mask2'].astype(bool))&mode_1_LF_mask
ds_fs['A1_LF_hb_id'] = ds_fs['A1_LF_hb'].where(mode_1_mask)

### Mode 2 ID
ds_T['m2_upper_iso'] = xr.DataArray(signal.sosfiltfilt(filt,ds_T['m2_upper_iso'].values),dims=['time'],coords={'time':ds_T.time.values})
ds_T['m2_lower_iso'] = xr.DataArray(signal.sosfiltfilt(filt,ds_T['m2_lower_iso'].values),dims=['time'],coords={'time':ds_T.time.values})
ds_fs['m2_upper_iso'] = ds_T['m2_upper_iso'].interp_like(ds_fs['Temp_Dave']) 
ds_fs['m2_lower_iso'] = ds_T['m2_lower_iso'].interp_like(ds_fs['Temp_Dave']) 
phase_mask = (ds_fs['m2_upper_iso'] > phase_upper_thresh)&(ds_fs['m2_lower_iso'] < phase_lower_thresh)
m2_mask= ((ds_fs['A2_LF_hb'] > mode_2_LF_thresh)&phase_mask).rolling(time=max_window,min_periods=5,center=True).max().astype(bool)
m2_mask = m2_mask&(~ds_fs['bore_mask2'].astype(bool))&mode_2_LF_mask&~mode_2_false_id
ds_fs['A2_LF_hb_id'] = ds_fs['A2_LF_hb'].where(m2_mask)

### HF wave ID
HF_mask = ((ds_fs['A1_HF_hb'] > mode_1_HF_thresh)).rolling(time=10,min_periods=1,center=True).max().astype(bool)
HF_mask = HF_mask&(~ds_fs['bore_mask2'].astype(bool))&mode_1_HF_mask&(~mode_1_mask)&(~m2_mask)
ds_fs['A1_HF_hb_id'] = ds_fs['A1_HF_hb'].where(HF_mask)

# Site Conditions Overview

In [ ]:
M1_oc = 100*np.nansum(~np.isnan(ds_fs.A1_LF_hb_id.values))/len(ds_fs.time.values)
M2_oc = 100*np.nansum(~np.isnan(ds_fs.A2_LF_hb_id.values))/len(ds_fs.time.values)
HF_oc = 100*np.nansum(~np.isnan(ds_fs.A1_HF_hb_id.values))/len(ds_fs.time.values)
bores_oc = 100*np.nansum(~np.isnan(ds_fs.bottom_hb.where(ds_fs.bore_mask).values))/len(ds_fs.time.values)

# Percent occurrence of each wave type
ds_fs['M1_LF_occ'] =  xr.DataArray(M1_oc)
ds_fs['M1_HF_occ'] =  xr.DataArray(HF_oc)
ds_fs['M2_occ'] =  xr.DataArray(M2_oc)
ds_fs['bores_occ'] =  xr.DataArray(bores_oc)

# Contributions

In [ ]:
### total mixing
ds_fs['total_flux'] = ds_fs['JQ'].sum(dim='time',skipna=True)

## Bore Contribution
ds_fs['bore_cont'] = 100*ds_fs['JQ'].where(ds_fs['bore_mask']).sum(dim='time',skipna=True)/ds_fs['total_flux']

## Mode 2 Contribution
ds_fs['M2_cont'] = 100*ds_fs['JQ'].where(~np.isnan(ds_fs['A2_LF_hb_id'])).sum(dim='time',skipna=True)/ds_fs['total_flux']

## Mode 1 HF Mixing
ds_fs['M1_HF_cont'] = 100*ds_fs['JQ'].where(~np.isnan(ds_fs['A1_HF_hb_id'])).sum(dim='time',skipna=True)/ds_fs['total_flux']

## Mode 1 LF Mixing
ds_fs['M1_LF_cont'] = 100*ds_fs['JQ'].where(~np.isnan(ds_fs['A1_LF_hb_id'])).sum(dim='time',skipna=True)/ds_fs['total_flux']

# AM and Medians

In [ ]:
ds_fs['JQ_t_mean'] =  ds_fs['JQ'].mean(dim=['time'],skipna=True)
ds_fs['JQ_t_median'] =  ds_fs['JQ'].median(dim=['time'],skipna=True)

In [ ]:
ds_fs.to_netcdf('..\Data\Processed\RS2019_T200_FS_v14.nc')
ds_T.to_netcdf('..\Data\Processed\RS2019_T200_IW_v14.nc')